# Install Necessary packages

In [0]:
!pip install pytorch_pretrained_bert
!pip install torch
!pip install transformers
!pip install barbar
!pip install pickle

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'gpt2data/'
import os
import sys
sys.path.append(base_dir)
print(os.getcwd())
print(os.listdir())
os.chdir(base_dir)

Mounted at /content/gdrive
/content
['.config', 'gdrive', 'sample_data']


# Imports

In [0]:
from __future__ import unicode_literals

import pandas as pd
from barbar import Bar
import math
import torch
from transformers import GPT2Tokenizer, GPT2DoubleHeadsModel, AdamW
from transformers import BertTokenizer, BertForSequenceClassification
from itertools import chain
import json
import pickle
import torch.nn as nn
import os
import re
import unicodedata
import torch.optim as optim
import torch.nn.functional as F
import random
from torch.utils.data import Dataset as Dataset
from torch.utils.data import DataLoader, random_split
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import confusion_matrix
import seaborn as sn
import numpy as np
import matplotlib.pyplot as plt

from transformers import cached_path
import tarfile
import tempfile
import logging

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
print(device)

cuda


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Initialise GPT-2 Model and tokenizer with pretrained weights

In [0]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2DoubleHeadsModel.from_pretrained('gpt2')

# Add special tokens to embeddings and vocabulary of the model
SPECIAL_TOKENS = ['<BOS>', '<EOS>', '<speaker1>', '<speaker2>', '<PAD>']
special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>',
                       'additional_special_tokens': ['<speaker1>', '<speaker2>']}  # ,'speaker2' : '<speaker2>']}

num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))
model.to(device)

GPT2DoubleHeadsModel(
  (transformer): GPT2Model(
    (wte): Embedding(50262, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2):

# Dataset Creation

In [0]:
datafile = 'dialogues_text2.txt'
emodatafile = 'dialogues_emotion2.txt'

bos, eos, speaker1, speaker2, CLS  = "<BOS>", "<EOS>", "<speaker1>", "<speaker2>", "<CLS>"

def normalizeString(s): #normailizes string
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

def filtertomaxlength(pair,emotionpair): #makes the max string have 200 words
    pair1 = []
    pair2 = []
    for i in range(len(pair)):
        length = 0
        for j in range(len(pair[i])):
            length += len(pair[i][j].split(' '))
        
        if length < 51:
            pair1.append(pair[i])
            pair2.append(emotionpair[i])
    
    return pair1,pair2


def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )



def create_dialogues2(datafile,emodatafile): #takes text and puts it into a list 
    emopairs = []
    lines = open(datafile, encoding='utf-8').\
            read().strip().split('\n')
            
    emolines = open(emodatafile, encoding='utf-8').\
            read().strip().split('\n')
        # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('__eou__')] for l in lines]
    for i in range(len(pairs)):
        pairs[i] = pairs[i][:-1] #removes empty character and keeps rest
        emopairs.append(emolines[i].split(' '))
        emopairs[-1] = emopairs[-1][:-1]
        if len(emopairs[i]) != len(pairs[i]):
            print(i)
            print(len(emopairs[i]))
            print(len(pairs[i]))
            print(emopairs[i])
            print(pairs[i])
    return pairs,emopairs





def createdata(pairs,emotionpairs):
    output = []
    emotions = []
    for line in range(len(pairs)):        
        for j in range(len(pairs[line])-1):
            if (len(tokenizer.encode(pairs[line][j])) + len(tokenizer.encode(pairs[line][j+1]))) <= 51:
                output.append([pairs[line][j],pairs[line][j+1]])
                emotions.append([emotionpairs[line][j],emotionpairs[line][j+1]])
    return output,emotions


def balancedata(pairs,emotionpairs, maxbal):
    distribution = [0,0,0,0,0,0,0]

    for labely in range(7):
        for count in emotionpairs:
            #print(count)
            #print(labely)
            if int(count[1]) == labely:
                #print('yes')
                distribution[labely] += 1
    
    print(distribution)


    newpairs = []
    newemotions = []
    distribution = [0,0,0,0,0,0,0]

    for count in range(len(emotionpairs)):
        if distribution[int(emotionpairs[count][1])] < maxbal:
                distribution[int(emotionpairs[count][1])] += 1
                newpairs.append(pairs[count])
                newemotions.append(emotionpairs[count])
    
    print(len(newemotions))


    distribution = [0,0,0,0,0,0,0]
    
    for labely in range(7):
        for count in newemotions:
            #print(count)
            #print(labely)
            if int(count[1]) == labely:
                #print('yes')
                distribution[labely] += 1
    
    print(distribution)
    
    
    
    return newpairs,newemotions




pairs,emotionpairs = create_dialogues2(datafile,emodatafile)

print(len(pairs), len(emotionpairs))

pairs,emotionpairs = createdata(pairs,emotionpairs)


print(len(pairs), len(emotionpairs))

pairs,emotionpairs  = balancedata(pairs,emotionpairs,2000)

print(len(emotionpairs))
print(len(pairs))

14118 14118
96117 96117
[75396, 1756, 674, 491, 12842, 1598, 3360]
10519
[2000, 1756, 674, 491, 2000, 1598, 2000]
10519
10519


# Adds vocab not in GPT2

In [0]:
# this block adds vocab no in GPT2 to gpt2
vocab = []
for x in range(len(pairs)):
    for j in range(len(pairs[x])):
        words = pairs[x][j].split(' ')
        for w in words:
            if w not in vocab:
                vocab.append(w)
tokenizer.add_tokens(vocab)
model.resize_token_embeddings(len(tokenizer))

Embedding(55805, 768)

# Functions to get data into Persona, History, Reply format along with Input_IDs, Token_Type_IDs,MC_Token IDs, Language Modelling and Multiple Choice labels,

In [0]:

# Build Inputs and Create_rest_of_dataset change a question answer pair into appropriate format to feed into model
def build_inputs(persona, history, reply):
    sequence = [[bos] + list(chain(*persona))] + history + [reply + [eos]]
    sequence = [sequence[0]] + [[speaker2 if (len(sequence) - i) % 2 else speaker1] + s
                                for i, s in enumerate(sequence[1:])]
    # Build our word, segments and position inputs from the sequence
    words = list(chain(*sequence))  # word tokens
    segments = [speaker2 if i % 2 else speaker1  # segment tokens
                for i, s in enumerate(sequence) for _ in s]
    position = list(range(len(words)))  # position tokens
    return words, segments, position, sequence


def create_instance(personality, history, reply):
    # print('histor in create rest of dataset', history)
    seq1 = []
    for i in history:
        seq1.append(re.sub("[^\w]", " ", i).split())

    seq0 = personality

    seq2 = re.sub("[^\w]", " ", reply).split()
    words, segments, position, sequence = build_inputs(seq0, seq1, seq2)

    # gold reply

    words = tokenizer.convert_tokens_to_ids(words)
    segments = tokenizer.convert_tokens_to_ids(segments)

    lm_targets = ([-100] * sum(len(s) for s in sequence[:-1])) \
                 + [-100] + tokenizer.convert_tokens_to_ids(sequence[-1][1:])
    last_token = len(words) - 1

    # distractor creation

    words_distractor_list = []
    segments_distractor_list = []
    lm_distractor_list = []
    last_token_distractor_list = []

    emotionscovered = [0, 0, 0, 0, 0, 0, 0]
    emotionscovered[int(personality)] = 1

    for numdistractors in range(7):
        check = 1
        while check == 1:
            distractor = random.choice(range(len(pairs)))
            if emotionscovered[numdistractors] == 0:
                if int(emotionpairs[distractor][-1]) == numdistractors:
                    emotionscovered[numdistractors] = 1
                    check = 0
                    distractor = pairs[distractor][-1]

                    distractor = re.sub("[^\w]", " ", distractor).split()
                    words_distractor, segments_distractor, _, _ = build_inputs(seq0, seq1, distractor)
                    words_distractor = tokenizer.convert_tokens_to_ids(words_distractor)
                    segments_distractor = tokenizer.convert_tokens_to_ids(segments_distractor)
                    lm_distractor = [-100] * len(words_distractor)
                    last_token_distractor = len(words_distractor) - 1

                    words_distractor_list.append(words_distractor)
                    segments_distractor_list.append(segments_distractor)
                    lm_distractor_list.append(lm_distractor)
                    last_token_distractor_list.append(last_token_distractor)
            else:
                check = 0

    padding_length = 100

    (words, segments) = [pad(x, tokenizer.convert_tokens_to_ids('<PAD>'), padding_length)
                         for x in (words, segments)]

    for m in range(len(words_distractor_list)):
        (words_distractor_list[m], segments_distractor_list[m]) = [
            pad(x, tokenizer.convert_tokens_to_ids('<PAD>'), padding_length)
            for x in (words_distractor_list[m], segments_distractor_list[m])]
        lm_distractor_list[m] = pad(lm_distractor_list[m], -100, padding_length)

    lm_targets = pad(lm_targets, -100, padding_length)

    ###########################
    inputs_list = [words]
    tokentype_list = [segments]
    mctoken_list = [last_token]
    lmlabel_list = [lm_targets]

    for numcand in range(6):
        inputs_list.append(words_distractor_list[numcand])
        tokentype_list.append(segments_distractor_list[numcand])
        mctoken_list.append(last_token_distractor_list[numcand])
        lmlabel_list.append(lm_distractor_list[numcand])

    input_ids = torch.tensor(inputs_list, dtype=torch.long)
    token_type_ids = torch.tensor(tokentype_list, dtype=torch.long)
    mc_token_ids = torch.tensor(mctoken_list, dtype=torch.long)
    lm_labels = torch.tensor(lmlabel_list, dtype=torch.long)
    mc_labels = torch.tensor([0], dtype=torch.long)

    return input_ids, token_type_ids, mc_token_ids, lm_labels, mc_labels


# Evaluation data tokenizer

def build_input_from_segments(persona, history, reply, tokenizer, lm_labels=False, with_eos=True):
    sequence = [[bos] + list(chain(*persona))] + history + [reply + ([eos] if with_eos else [])]
    sequence = [sequence[0]] + [[speaker2 if (len(sequence) - i) % 2 else speaker1] + s for i, s in
                                enumerate(sequence[1:])]
    instance = {}
    instance["input_ids"] = list(chain(*sequence))
    instance["token_type_ids"] = [speaker2 if i % 2 else speaker1 for i, s in enumerate(sequence) for _ in s]
    instance["mc_token_ids"] = len(instance["input_ids"]) - 1
    instance["lm_labels"] = [-100] * len(instance["input_ids"])
    if lm_labels:
        instance["lm_labels"] = ([-100] * sum(len(s) for s in sequence[:-1])) + [-100] + sequence[-1][1:]
    return instance


def pad(x, padding, padding_length):
    return x + [padding] * (padding_length - len(x))


# Creating Dataloader for train, validation and test set

In [0]:
class Pairsdataset(Dataset):
    def __init__(self, pairs,emotionpairs):
        
        self.pairs = pairs
        self.emotionpairs = emotionpairs
        
        self.valuelist = []
        
        
        for i in range(len(self.pairs)):
            #print(i)
            instance = create_instance(self.emotionpairs[i][-1],self.pairs[i][:-1],self.pairs[i][-1])
            self.valuelist.append(instance)
                
             
        maxl = 0
        
        for i in range(len(self.valuelist)):
            
            if len(self.valuelist[i][0][0]) > maxl:
                maxl = len(self.valuelist[i][0][0])
                f = i
        print(maxl,f)
        
        
    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, i):
      
        input_ids, token_type_ids, mc_token_ids, lm_labels, mc_labels = self.valuelist[i]
        return input_ids, token_type_ids, mc_token_ids, lm_labels, mc_labels

In [0]:
Train_dataset = Pairsdataset(pairs,emotionpairs)

train, valid,test = torch.utils.data.random_split(Train_dataset, [9000,1000,519])


train_loader = DataLoader(train, batch_size= 2, shuffle=True, num_workers=0)
valid_loader = DataLoader(valid, batch_size= 2, shuffle=False, num_workers=0)
test_loader = DataLoader(test, batch_size= 1, shuffle=False, num_workers=0)

#Creating testing dataset

test_list = []
for x in range(len(test)):
    sentence = ""
    
    for i in range(len(test[x][0][0])):
        
        if test[x][0][0][i] == 50261:
            s = i + 1
        if test[x][0][0][i] == 50260:
            e = i
    
    tokens = test[x][0][0][s:e]
    
    
    for chars in tokens:
        sentence = sentence + (tokenizer.convert_ids_to_tokens([chars])[0]) + ' ' 
        
        
    test_list.append(sentence[:-1])

print(len(test_list))



100 0
519


# Setting up optimizer and training function

In [0]:
learning_rate = 6.25e-5
optimizer = AdamW(model.parameters(), lr=6.25e-5, correct_bias=True)
epochs = 3
total_steps = len(train_loader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

lm_coef = 1.0
mc_coef = 1.0


print(len(train_loader))

4500


In [0]:
output_dir = 'gpt2_model/'
def train_batch():
    grad_acc = 8
    lm_coef = 1.0
    mc_coef = 1.0
    minloss = 100
    loss_analytics = []
    lm_loss_analytics = []
    mc_loss_analytics = []
    valloss_analytics = []
    current_loss_analytics = []
    current_lm_loss_analytics = []
    current_mc_loss_analytics = []
    perplexity = []
    vallm_loss_analytics = []
    valmc_loss_analytics = []
    valnll_analytics = []
    iterations = []
    output_dir = 'gpt2_model/'
    total_loss = 0
    total_lmloss = 0
    total_mcloss = 0
    total_val_loss = 0
    total_val_lmloss = 0
    total_val_mcloss = 0
    for i in range(epochs):        
        print('Epoch number', i)
        for id, X in enumerate(Bar(train_loader)):
            model.train()
            optimizer.zero_grad()

            input_ids, token_type_ids, mc_token_ids, lm_labels, mc_labels = X

            input_ids = input_ids.to(device)
            # print(input_ids)
            # print(token_type_ids)
            token_type_ids = token_type_ids.to(device)
            mc_token_ids = mc_token_ids.to(device)
            lm_labels = lm_labels.to(device)
            mc_labels = mc_labels.to(device)

            (lm_loss), (mc_loss), *_ = model(
            input_ids, token_type_ids=token_type_ids, mc_token_ids=mc_token_ids,
            mc_labels=mc_labels, lm_labels=lm_labels)

            loss = lm_loss * lm_coef + mc_loss * mc_coef
            current_loss_analytics.append(loss)
            current_lm_loss_analytics.append(lm_loss)
            current_mc_loss_analytics.append(mc_loss)

            total_loss += loss.item()
            total_lmloss += lm_loss.item()
            total_mcloss += mc_loss.item()
            iteration = (i * len(train_loader)) + id + 1
            loss_analytics.append(total_loss / iteration)
            lm_loss_analytics.append(total_lmloss / iteration)
            mc_loss_analytics.append(total_mcloss / iteration)
           


            loss.backward()

            _ = nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            if id % 200 == 0:
                iterations.append(iteration)
                total_val_loss = 0
                total_val_lmloss = 0
                total_val_mcloss = 0
                model.eval()
                for id, batch in enumerate(Bar(valid_loader)):
                    input_ids, token_type_ids, mc_token_ids, lm_labels, mc_labels= batch
                    input_ids = input_ids.to(device)
                    # print(input_ids)
                    # print(token_type_ids)
                    token_type_ids = token_type_ids.to(device)
                    mc_token_ids = mc_token_ids.to(device)
                    lm_labels = lm_labels.to(device)
                    mc_labels = mc_labels.to(device)
                    with torch.no_grad():
                        (lm_loss), (mc_loss), *_ = model(input_ids, token_type_ids=token_type_ids, mc_token_ids=mc_token_ids, mc_labels=mc_labels, lm_labels=lm_labels)
                        loss = lm_loss * lm_coef + mc_loss * mc_coef
                        total_val_loss += loss.item()
                        total_val_lmloss += lm_loss.item()
                        total_val_mcloss += mc_loss.item()
                perplexity.append(math.exp(total_val_lmloss/len(valid_loader)))        
                vallm_loss_analytics.append(total_val_lmloss / len(valid_loader))
                valmc_loss_analytics.append(total_val_mcloss / len(valid_loader))
                valloss_analytics.append(total_val_loss / len(valid_loader))
    with open('loss_analytics', 'wb') as f:
        pickle.dump(loss_analytics, f)
    with open('lm_loss_analytics', 'wb') as f:
        pickle.dump(lm_loss_analytics, f)
    with open('mc_loss_analytics', 'wb') as f:
        pickle.dump(mc_loss_analytics, f)
    with open('valloss_analytics', 'wb') as f:
        pickle.dump(valloss_analytics, f)
    with open('vallm_loss_analytics', 'wb') as f:
        pickle.dump(vallm_loss_analytics, f)
    with open('valmc_loss_analytics', 'wb') as f:
        pickle.dump(valmc_loss_analytics, f)
    with open('current_loss_analytics', 'wb') as f:
        pickle.dump(current_loss_analytics, f)
    with open('current_lm_loss_analytics', 'wb') as f:
        pickle.dump(current_lm_loss_analytics, f)
    with open('current_mc_loss_analytics', 'wb') as f:
        pickle.dump(valmc_loss_analytics, f)
    with open('iterations', 'wb') as f:
        pickle.dump(iterations, f)
    model.save_pretrained(save_directory=output_dir)
    tokenizer.save_pretrained(save_directory=output_dir)
    

train_batch()
    

Epoch number 0
1000/1000: [===============================>] - ETA 0.1s
1000/1000: [===============================>] - ETA 0.1s
1000/1000: [===============================>] - ETA 0.1s
1000/1000: [===============================>] - ETA 0.1s
1000/1000: [===============================>] - ETA 0.1s
1000/1000: [===============================>] - ETA 0.1s
1000/1000: [===============================>] - ETA 0.1s
1000/1000: [===============================>] - ETA 0.1s
1000/1000: [===============================>] - ETA 0.1s
1000/1000: [===============================>] - ETA 0.1s
1000/1000: [===============================>] - ETA 0.1s
1000/1000: [===============================>] - ETA 0.1s
1000/1000: [===============================>] - ETA 0.1s
1000/1000: [===============================>] - ETA 0.1s
1000/1000: [===============================>] - ETA 0.1s
1000/1000: [===============================>] - ETA 0.1s
1000/1000: [===============================>] - ETA 0.1s
1000/1000: [====

/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


# Evaluation of Model using Sentiment Analysis

In [0]:
!pip install transformers
from transformers import GPT2LMHeadModel

def top_filtering(logits, top_k=0., top_p=0.9, threshold=-float('Inf'), filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k, top-p (nucleus) and/or threshold filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k: <=0: no filtering, >0: keep only top k tokens with highest probability.
            top_p: <=0.0: no filtering, >0.0: keep only a subset S of candidates, where S is the smallest subset
                whose total probability mass is greater than or equal to the threshold top_p.
                In practice, we select the highest probability tokens whose cumulative probability mass exceeds
                the threshold top_p.
            threshold: a minimal threshold to keep logits
    """
    assert logits.dim() == 1  # Only work for batch size 1 for now - could update but it would obfuscate a bit the code
    top_k = min(top_k, logits.size(-1))
    if top_k > 0:
        # Remove all tokens with a probability less than the last token in the top-k tokens
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        # Compute cumulative probabilities of sorted tokens
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probabilities = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probabilities > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        # Back to unsorted indices and set them to -infinity
        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value

    indices_to_remove = logits < threshold
    logits[indices_to_remove] = filter_value

    return logits


#%%

history = []
reply = ""
first = 1
sent = ""


temperature = 0.7
decoding_strategy = "nucleus" # choose between greedy, beam, top-k, nucleus

replies = []
prompts = []


for i in test_list:
        
        #initialize bert 
        tokenizer = BertTokenizer.from_pretrained('/Sentiment Analysis/')
        model = BertForSequenceClassification.from_pretrained('./Sentiment Analysis/',num_labels = 5, output_attentions = False, output_hidden_states = False)
        model.to(device)
        
        
        model.eval()
        
        
        
        if first !=1:
            #This is for getting the emotion of our previous reply
            ############################
            encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 78,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )
            
            
            input_ids = [encoded_dict['input_ids']]
            attention_masks = [encoded_dict['attention_mask']]

            input_ids = torch.cat(input_ids, dim=0)
            attention_masks = torch.cat(attention_masks, dim=0)

            input_ids = input_ids.to(device)
            attention_mask = attention_masks.to(device)

            outputs = model(input_ids,token_type_ids=None,attention_mask = attention_mask)
            values, indices = torch.max(outputs[0][0],0)

            personality = indices.item()
            #print('Reply personality', personality, personalities[personality])
            
            replies.append(personality)
        ###############################
        first = 0
        
        
        
        #get our input
        
        raw_text = i
        
        
        # This is for getting the emotion of our input
        ############################
        encoded_dict = tokenizer.encode_plus(
                        raw_text,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 78,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
        
        input_ids = [encoded_dict['input_ids']]
        attention_masks = [encoded_dict['attention_mask']]
        
        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)
        
        input_ids = input_ids.to(device)
        attention_mask = attention_masks.to(device)
        
        outputs = model(input_ids,token_type_ids=None,attention_mask = attention_mask)
        values, indices = torch.max(outputs[0][0],0)
        
        personality = indices.item()
        
        
        prompts.append(personality)
        
        #Shifts emotions from BERT to be same as trained for GPT-2
        if personality > 1:
            personality += 2
        
               
        #############################
        
        #Change to GPT-2 model, language modelling head only
        model = GPT2LMHeadModel.from_pretrained(output_dir).to(device)
        tokenizer = GPT2Tokenizer.from_pretrained(output_dir)
        
       
        seq0 = str(personality)
        seq1 = re.sub("[^\w]", " ",  raw_text).split()
        seq2 = re.sub("[^\w]", " ",  reply).split()
        
        history.append(re.sub("[^\w]", " ",  raw_text).split())
        #Writes reply according to decoding strategy
        while True:

            instance =  build_input_from_segments(seq0,history,seq2, tokenizer, lm_labels=False, with_eos= False)
            input_ids = tokenizer.convert_tokens_to_ids(instance["input_ids"])

            token_type_ids = tokenizer.convert_tokens_to_ids(instance["token_type_ids"])

            input_ids = torch.tensor([input_ids], dtype=torch.long)
            
            token_type_ids = torch.tensor([token_type_ids], dtype=torch.long)

            input_ids = input_ids.to(device)

            token_type_ids = token_type_ids.to(device)
            
             
            
            with torch.no_grad():
                outputs = model(input_ids=input_ids, token_type_ids = token_type_ids)
                
                
                
                predictions = outputs[0]
                
               
                ################
                #Changes based on decoding strategy 
                
                if decoding_strategy == "greedy":
                    value,greedy = torch.max(predictions[0, -1, :],0)
                    prev = greedy
    
                elif decoding_strategy == "top-k":
            
                    predictions = predictions[0, -1, :]/ temperature
                    predictions = top_filtering(predictions, top_k=20, top_p= 0.0)
                    probs = F.softmax(predictions, dim = -1)
                    prev = torch.multinomial(probs, 1)
                    
    
                elif decoding_strategy == "nucleus":
                
                    predictions = predictions[0, -1, :]/ temperature
                    predictions = top_filtering(predictions, top_k=0, top_p=0.9)
                    probs = F.softmax(predictions, dim = -1)
                    prev = torch.multinomial(probs, 1)   
                    
                
                predicted_text = tokenizer.decode([prev.item()])
                if predicted_text in SPECIAL_TOKENS:
                    x = 1
                else:
                    seq2.append(predicted_text)
                    
                    
            if predicted_text == '<EOS>' and len(seq2)>0:
                history.append(seq2)
                break
        
                
        
        history = []
        sent = ""
        for i in range(len(seq2)):
            sent = sent + str(seq2[i]) + ' '
        
         
#Gets confusion matrix
prompts = prompts[:-1]
cm = confusion_matrix(prompts, replies)
cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

totalcorrects = 0


for i in range(len(replies)):
    if replies[i] == prompts[i]:
        totalcorrects += 1

        
print(cmn)
print(totalcorrects/(len(replies)))  
print(confusion_matrix(prompts, replies))
sn.heatmap(cmn,cmap = 'Reds', annot=True)  
plt.ylabel('Detected Emotion of Prompt')
plt.xlabel('Detected Emotion of Reply')